In [1]:
# pip install requests

In [2]:
import requests
import json
import os
from requests.utils import quote
import threading
from queue import Queue

In [3]:
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

In [4]:
GITHUB_REPO = "ashishchauhan0001/CampusNest"
GITHUB_TOKEN = os.getenv("GITHUB_PAT")
HEADERS = {"Authorization": f"token {GITHUB_TOKEN}"}

In [5]:
def get_default_branch():
    url = f"https://api.github.com/repos/{GITHUB_REPO}"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json().get("default_branch", "main")
    return "main"

DEFAULT_BRANCH = get_default_branch()
print(f"Default branch: {DEFAULT_BRANCH}")

Default branch: main


In [6]:
def get_repo_contents(path=""):
    url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{quote(path)}"
    response = requests.get(url, headers=HEADERS)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"[ERROR] Failed to fetch {url}: {response.status_code}")
        return []

In [7]:
def get_file_content(file_path):
    url = f"https://raw.githubusercontent.com/{GITHUB_REPO}/{DEFAULT_BRANCH}/{quote(file_path)}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.text
    return None

In [8]:
file_data_lock = threading.Lock()
dependencies_lock = threading.Lock()

In [9]:
def analyze_repo(path="", max_threads=8):
    repo_structure = get_repo_contents(path)
    file_data = []
    dependencies = []
    
    skip_files = {
        ".gitignore", "README.md", "LICENSE", ".dockerignore",
        ".env", ".editorconfig", "package-lock.json", "node_modules"
    }
    skip_extensions = {
        ".png", ".jpg", ".jpeg", ".gif", ".ico", ".pdf", ".svg",
        ".zip", ".tar", ".gz", ".exe", ".dll", ".so", ".jar",
        ".bin", ".lock", ".mp4", ".mkv", ".mp3", ".m3u8", ".mov",
        ".avi", ".flv", ".webm", ".wav", ".aac", ".ogg", ".opus",
        ".m4a", ".3gp", ".3g2", ".wma", ".doc", ".docx", ".xls",
        ".xlsx", ".ppt", ".pptx", ".odt", ".ods", ".odp", ".csv",
        ".json", ".xml", ".yaml", ".yml", ".html", ".css"
    }

    file_queue = Queue()
    
    def process_file_worker():
        while True:
            item = file_queue.get()
            if item is None:
                file_queue.task_done()
                break
                
            print(f"[INFO] Processing file: {item['path']}")
            name = item["name"]
            ext = "." + name.split(".")[-1] if "." in name else ""

            if name in skip_files or ext.lower() in skip_extensions:
                file_queue.task_done()
                continue

            file_content = get_file_content(item["path"])

            with file_data_lock:
                file_data.append({
                    "name": name,
                    "size": item["size"],
                    "html_url": item["html_url"],
                    "path": item["path"],
                    "content": file_content
                })

            if name in ["package.json", "pom.xml", "requirements.txt"]:
                with dependencies_lock:
                    dependencies.append(item["path"])
                    
            file_queue.task_done()

    threads = []
    for i in range(max_threads):
        t = threading.Thread(target=process_file_worker)
        t.start()
        threads.append(t)

    for item in repo_structure:
        if item["type"] == "dir":
            sub_file_data, sub_dependencies = analyze_repo(item["path"])
            file_data.extend(sub_file_data)
            dependencies.extend(sub_dependencies)
        elif item["type"] == "file":
            file_queue.put(item)

    file_queue.join()
    
    for i in range(max_threads):
        file_queue.put(None)
    for t in threads:
        t.join()

    return file_data, dependencies

In [10]:
def generate_report():
    files, dep_files = analyze_repo()
    dependencies = {file: get_file_content(file) for file in dep_files}

    report = {
        "repo": GITHUB_REPO,
        "total_files": len(files),
        # "largest_files": sorted(files, key=lambda x: x["size"], reverse=True)[:5],
        "files": files,
        "dependencies": dependencies
    }

    repo_scan_report_path = os.path.join(output_dir, "repo_scan_report.json")
    with open(repo_scan_report_path, "w") as f:
        json.dump(report, f, indent=4)

    print("[INFO] Scan Completed: Report saved as repo_scan_report.json")

In [11]:
generate_report()

[INFO] Processing file: .gitignore[INFO] Processing file: README.md

[INFO] Processing file: api/controllers/auth.controller.js[INFO] Processing file: api/controllers/listing.controller.js
[INFO] Processing file: api/controllers/razorpay.controller.js
[INFO] Processing file: api/controllers/request.controller.js
[INFO] Processing file: api/controllers/review.controller.js
[INFO] Processing file: api/controllers/tenant.controller.js
[INFO] Processing file: api/controllers/user.controller.js
[INFO] Processing file: api/controllers/vendor.controller.js

[INFO] Processing file: api/index.js
[INFO] Processing file: api/models/listing.model.js[INFO] Processing file: api/models/request.model.js
[INFO] Processing file: api/models/review.model.js
[INFO] Processing file: api/models/tenant.model.js
[INFO] Processing file: api/models/user.model.js
[INFO] Processing file: api/models/vendor.model.js

[INFO] Processing file: api/routes/auth.route.js
[INFO] Processing file: api/routes/listing.route.js